In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import config
import subprocess
import mediapipe as mp
from utils import load_and_preprocess_images, create_model
import joblib
import cv2
import numpy as np
from model_train import train_model

In [ ]:
model = create_model(input_size= [64,64,1], loss= "categorical_crossentropy", config= config)
model_train = train_model(model= model, config = config)

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect and recognize using webcam
def recognize_person(model, target_size=(64, 64)):
    cap = cv2.VideoCapture(0)  # 0 for default camera

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale and resize
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x,y,w,h) in faces : 
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green rectangle around the face

            # Extract the face region for recognition
            face_region = gray_frame[y:y + h, x:x + w]
            resized_frame = cv2.resize(gray_frame, target_size)
            normalized_frame = resized_frame / 255.0
            reshaped_frame = normalized_frame.reshape(1, target_size[0], target_size[1], 1)

            # Make a prediction
            prediction = model.predict(reshaped_frame)
            print(prediction)

            # Decode the result (argmax to get class index)
            predicted_class = np.argmax(prediction)
            print(predicted_class)
            label = 'amisha' if predicted_class == 0 else 'Others'

            #Show results on the frame
            cv2.putText(frame, label, (x,y -10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            if label == 'Person':
                print("Face recognized!! Running the virtual mouse program....")
                break
        
        cv2.imshow('Face Recognition', frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if label == "Person":
        subprocess.run(["python", "gesture_control.py"])


In [ ]:
recognize_person(model)